In [1]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [2]:
!python --version

Python 3.8.3


In [3]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook

In [4]:
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
# query_txt2 = input('2.제외할 첫번째 키워드를 입력하세요: ')
# query_txt3 = input('3.제외할 두번째 키워드를 입력하세요: ')

start_date = input('4.조회를 시작할 날짜를 입력하세요(예:20170101) :')
end_date = input('5.조회를 종료할 날짜를 입력하세요(예:20171231): ')

1.크롤링할 키워드는 무엇입니까?: 테슬라
4.조회를 시작할 날짜를 입력하세요(예:20170101) :20200101
5.조회를 종료할 날짜를 입력하세요(예:20171231): 20200105


In [45]:
query_txt="김치볶음밥 맛집"
start_date="20201201"
end_date="20210106"

# 크롤링할 블로그 url 수집하기

In [9]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

In [10]:
# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '검색옵션' 클릭
driver.find_element_by_link_text("검색옵션").click( )
time.sleep(1)

# Step 3. 정렬 : "관련도순" 
# 개발자 도구에서 정렬 버튼의 id 가 보이지 않습니다.
# 이럴 경우 쉽게 사용할 수 있는 방법이 바로 xpath 를 이용하는 방법입니다.
# xpath는 개발자 도구에서 해당 메뉴 부분을 마우스 오른쪽 버튼을 누르고 
# copy -> copy xpath 를 선택하면 됩니다
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click( )  # 정렬 버튼의 xpath 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click( ) # 관련도순 xpath
time.sleep(1)

# Step 4. 날짜 입력
driver.find_element_by_link_text("기간").click()
time.sleep(2)

# 이 부분이 아주 중요합니다.
# 날짜 부분에 날짜를 입력할 때 입력 속도가 너무 빠를 경우 날짜가 입력이 되다가 
# 오타가 나오는 경우가 많습니다.
# 그래서 날짜를 입력할 때 for 반복문을 사용해서 1 글자씩 입력하도록 코딩했습니다.

# 시작 날짜 입력하기
s_date = driver.find_element_by_id("date_from_input")
driver.find_element_by_id("date_from_input").click()
s_date.clear( )  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력하는 부분입니다.
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)
time.sleep(1)

# 종료 날짜 입력하기
e_date = driver.find_element_by_id("date_to_input")
driver.find_element_by_id("date_to_input").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)
    
# Step5. 날짜 입력 "적용" 버튼을 클릭 합니다.  
driver.find_element_by_class_name("tx").click()
time.sleep(3)

# # Step 8. 상세 검색버튼을 클릭 후 제외할 단어들을 설정합니다.
# ele2 = driver.find_element_by_id("inpop3")
# ele2.send_keys(query_txt2)
# ele2.send_keys(',')
# ele2.send_keys(query_txt3)
# driver.find_element_by_css_selector(".btn_ft.ty_green._search").click( )

## 글 url 크롤링 시작

In [11]:
# 스크롤 다운 

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 10
i = 0
while i < n:
    scroll_down(driver) # 스크롤 다운
    i += 1

In [12]:
url_list = []
title_list = []

# URL 크롤링 시작
articles = ".api_txt_lines.total_tit"
article_raw = driver.find_elements_by_css_selector(articles)

# url 크롤링 시작    
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))

테슬라 모델3 시승기 (롱레인지 듀얼모터 AWD)
테슬라 모델3_단 돈 3천만원대에 스웩 장착 후기
테슬라 모델3 짧은 시승 후기 (하남 스타필드)
[VicsunM3]테슬라 모델3 사고 및 수리기 1
테슬라 모델3 레인보우썬팅 시공후기~
배터리 수급 문제 : 테슬라의 Key Advantage
중국이 테슬라를 받아들이는 것은 전략적 옵션을 구축하는...
#15. 주식 : 테슬라(TSLA), 전기차 대장주!
테슬라 모델X PPF - 시공 걱정을 잘라드립니다
[네오튠즈]테슬라 모델3 레이블럭 열차단썬팅 시공
테슬라 모델s 서울 강남 파르나스 수퍼차저 첫 충전
BMW·토요타·GM, 테슬라 제압 나선 올드 스쿨의 반란...
테슬라 모델3 장기렌트 보조금 적용 조건 비교 방법
[쏘카 페어링] 테슬라 모델S 이용후기
제네시스 gv80 가격 vs 테슬라 모델x 말고 모델3 model3
요가복 후기 - 젝시믹스/뮬라웨어/스컬피그/STL/에르베/테슬라
테슬라 모델3 듀얼? 전동트렁크 시공으로 조금더 폼나게 그리고...
Canaccord가 테슬라 목표주가를 515달러로 올린 이유
수원유리막코팅 휠부터 유리까지 한번에 해결! 테슬라 모델3
엘론 머스크 테슬라 전기차 관하여 알아보자
전기자동차 슈퍼카 테슬라 모델3 미쉐린 파일럿알핀 pa4...
테슬라 모델3 신차유리막코팅... 작업일지-07 #부산테슬라광택...
[더비머] BMW 신형 3시리즈 G20 330i 테슬라모드...
테슬라 모델s 비 오는날 야외에서 첫 충전
성형외과 의사가 리뷰해 보았다 (feat.테슬라 모델3)
죽전썬팅 테슬라 모델3 후퍼옵틱 프리미엄나노세라믹+클라식...
테슬라 모델X, 오토파일럿 내비게이션 (자율주행, NOA)...
센트랄모텍 / 엠에스오토텍 : 테슬라, 상하이 생산 '모델 3...
테슬라 모델3 썬팅 완벽하게 마무리!
미국 내, 테슬라 오토파일럿 금지 여론
미국 내, 테슬라 오토파일럿 금지 여론
테슬라 모델3 카오디오 스피커 튜닝 feat. 크로스파이어
[더비머] BMW 신형 3시리즈 G20 330

현대기아자동차 gdi 보증
숙대입구맛집 남영돈에서 쫀득쫀득 가브리살 아삭아삭...
투자 결산단신 - 19년 12월
월스트리트 산책 2.0 - 2편: 월스트리트는 나쁜 남자들의 놀이터
프레스티지바이오로직스▶프레스티지바이오로직스 장외주식▶
2019년에 읽은 책들
화신테크 주가/주식/소개 (2019년 3분기 실적)
메뉴있음 분위기 좋은 이수역 맛집 미드나잇키친 심야식당
welcome to twenty ♥
[오늘의 한문장] 빌게이츠는 왜 과학책을 읽을까
미래차 : 삼성과 LG가 하길 기대했는데 화웨이랑 DJI가 더...
자주오개ㅡ애견카페.호텔.유치원
4끼 주는 서울 신라호텔 이그제큐티브 룸 라운지 패키지...
구래 맛집: 곱창+막창+대창에 염통, 홍창까지 즐길 수 있는...
[강남 역삼] 육즙 끝판왕 이베리코 돼지고기+ 메밀 막국수...
[광주맛집] 친구 단골맛집 양산동 '25시 참숯구이'
[Daily 주요기사] 2020-01-02
2020년 경기전망과 비전
6호선에 새로 개통된 최신 지상역- 서울 지하철 6호선 신내역...
영화 포드 v 페라리 '미국의 완승, 유럽은 죽었다.'
오늘의 주식시황 2020년 1월 3일 금요일 (상한가 - 센트랄모텍...
[주식마켓]오늘의 신문 돋보기! - 1월2일(목)
해외주식 투자노트 (2020.01.02)
이상한 너구리의 새해 인사 & 2019연말 결산
[제네시스 첫번째 SUV GV80] 이제 머지 않았다.

url갯수:  330
title갯수:  330


In [14]:
df = pd.DataFrame({'url':url_list, 'title':title_list})

# 저장하기
df.to_excel("blog_url.xlsx", encoding = 'utf-8-sig')

# 블로그 내용 크롤링하기

In [10]:
import sys
import os
import pandas as pd
import numpy as np

In [13]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("blog_url.xlsx") # 기본 모델

num_list = len(url_load)

print(num_list)
url_load

330


,Unnamed: 0,url,title
0,0,https://blog.naver.com/thearly?Redirect=Log&lo...,"홍대 배달 음식 맛집 [ 삼겹살김치볶음밥, 치킨마요덮밥 맛집..."
1,1,https://blog.naver.com/dnr6578?Redirect=Log&lo...,"[부산해운대구맛집/해운대맛집... 멘보샤, 김치볶음밥, 짬뽕"
2,2,https://blog.naver.com/i_believe28?Redirect=Lo...,[맛집/고터] 김치볶음밥이 맛있는 레트로 맛집 『하트타임』
3,3,https://blog.naver.com/woonelee?Redirect=Log&l...,[한남동 맛집] 알프키친... 항정살&대파구이 김치볶음밥...
4,4,https://blog.naver.com/queen05123?Redirect=Log...,서초동 맛집 호주 김치볶음밥이 대박이었던 교대 술집
...,...,...,...
325,325,https://blog.naver.com/happy_jennychoi?Redirec...,"[영국생활] 2020년 12월 런던일기 1, 런던맛집, 테이트브리튼"
326,326,https://blog.naver.com/gustkddndn?Redirect=Log...,추천해주는 청주 맛집리스트/ 구미사람이 추천해주는 구미 맛집
327,327,https://blog.naver.com/yurim012?Redirect=Log&l...,평택 법원 맛집 육풍에서 삼겹살 푸짐하게!
328,328,https://blog.naver.com/wwee4901?Redirect=Log&l...,"[쏨쏨이 여행 맛집] 괌여행 추천 찐 맛집 (도스버거, 에그앤띵스..."


In [17]:
dict = {}  # 전체 크롤링 데이터를 담을 그릇

# ★수집할 글 갯수 정하기
number = 30
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver.exe")  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    

        content_list = []
        for content in contents:
            content_list.append(content.text)           
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_content.xlsx")
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

0 홍대 배달 음식 맛집 [ 삼겹살김치볶음밥, 치킨마요덮밥 맛집 - 돈떡 ]
2 [맛집/고터] 김치볶음밥이 맛있는 레트로 맛집 『하트타임』
3 [한남동 맛집] 알프키친 / 믹스드 프라이즈 / 버터밀크 오믈렛 라이스 / 항정살&대파구이 김치볶음밥 / 화이트라구 바질 페스토 파스타 / 화이트와인
4 서초동 맛집 호주 김치볶음밥이 대박이었던 교대 술집
5 여수 분위기 맛집 바다식탁 : 갓김치볶음밥, 로제 바다파스타
6 찐제주도민맛집 빨강코끼리에서 돈까스쫄면, 새우김치볶음밥 먹었음
7 광화문 점심 맛집 도노베아토의 깔끔한 베이컨김치볶음밥!
8 영주맛집 ) 한솥도시락 새치고기고기, 스팸김치볶음밥, 소고기육개장 후기 : 한솥도시락 경북전문대점
9 [대구 맛집]매력적인 김치 볶음밥 "새한분식"
10 |성수맛집| '성수명당' 1등은 김치볶음밥!!! (ft.스테이크와 고등어초밥)
11 [대전술집/대전포차] 코다차야 -김치볶음밥 맛집
12 [부경대 맛집] 키다리식당, 부경대 김치볶음밥 맛집
13 의정부배달맛집 스테이크레이브 / 김치볶음밥과 파스타
14 대학생이추천하는 안양1번가 가성비좋은곳 김치볶음밥맛집 김볶스
15 배달의민족 헝그리박스에서 집밥같은 도시락 주문-! 차슈 낚지볶음 대패새우 + 김치볶음밥  + 치킨카레덮밥 헝그리박스추천메뉴 창원배달맛집
16 이천 포장배달 도시락 맛집 (스팸김치볶음밥/빅치킨마요/숯불삼겹정식&두부강된장)  '한솥도시락'
17 [청주 봉명동/맛집] 점심에 김치볶음밥이 땡긴다면? "아트쿡" 종합분식 최애 맛집
18 제주 서귀포 맛집 ‘취향의 섬’ 김치볶음밥 꼭 드세요 
19 중랑구 브런치 배달 맛집 이문동 카페115, 김치볶음밥 쿠팡이츠 배달 후기!
20 (여수맛집) 바다식탁 / 갓김치볶음밥 / 바다라면 / 페퍼로니피자
21 청담 맛집, 분위기 예쁜 카페 ‘미엘’ 김치볶음밥 디너
22 [일상일기] 청담/압구정 김치볶음밥 맛집? 와플맛집! '미엘 miel'
23 The장안부동산 동대문시장맛집 동대문시장짜볶이 김치볶음밥맛집 [남이분식]
24 노량진 맛집 저

Exception ignored in: <bound method tqdm.__del__ of   7%|█████▌                                                                             | 2/30 [10:01<07:22, 15.82s/it]>
Traceback (most recent call last):
  File "C:\Users\runia\Anaconda3\envs\py36\lib\site-packages\tqdm\_tqdm.py", line 931, in __del__
    self.close()
  File "C:\Users\runia\Anaconda3\envs\py36\lib\site-packages\tqdm\_tqdm.py", line 1133, in close
    self._decr_instances(self)
  File "C:\Users\runia\Anaconda3\envs\py36\lib\site-packages\tqdm\_tqdm.py", line 496, in _decr_instances
    cls.monitor.exit()
  File "C:\Users\runia\Anaconda3\envs\py36\lib\site-packages\tqdm\_monitor.py", line 52, in exit
    self.join()
  File "C:\Users\runia\Anaconda3\envs\py36\lib\threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread



수집한 글 갯수:  29
{0: {'title': '홍대 배달 음식 맛집 [ 삼겹살김치볶음밥, 치킨마요덮밥 맛집 - 돈떡 ]', 'nickname': 'EarLy\n(thearly)', 'datetime': '2021. 1. 4. 20:57', 'content': ' 요즘 사회적 거리두기\n적극 동참하며 \n음식은 댑부분 주문해서 먹고 \n나가질 않는다  친구가 없는게 아냐  암튼 \n나랑 짝꿍이\n가끔씩 시켜먹는 \n돈까스랑 떡볶이 전문점이지만\n김치볶음밥 맛집\n치킨마요 맛집 \nㅋㅋㅋ 돈떡 되시겠다. \nㅋㅋㅋ 이름부터 돈떡인데\n우리가 주문한건\nㅋㅋㅋㅋ ㅋㅋㅋㅋ\n삼겹살 김치볶음밥과 \n치킨마요 덮밥  구성이 뭐가 많다 ㅋㅋ  ㅋㅋ우동국물도 따뜻하게 온다 ㅋㅋ\n같이 온 종이컵에 따라서 마시라고 ㅋ 삼겹살 김치볶음밥 \n맛있는거 + 맛있는거\n= 더 맛있는거 \nㅋㅋㅋㅋㅋㅋㅋ 삼겹살이 이렇게 \n작게 썰려서 들어가있다 ㅋ 곱배기라서 \n양이 너무 많다 ㄷ ㄷ  윤기가 좌르르르 이건 \n치킨마요덮밥 치킨도 알차게 들어있다. \n다만 \n이것도 곱배기였는데 \n밥양에 비해서 \n소스 양이 조금 적었다. \n( 다음에 주문할때 말씀드려야겠다  ) 슥삭슥삭 비벼먹자 리뷰쓴다고 했더니 왔던 \n서비스 \n고구마 맛탕 맛탕도 맛이썽 요\n우동국물 ?? \n뭐지 그 장국 ?? \n뭐 여튼 뜨끈하게 오는 이 \n국물이 또 한몫한다  김치볶음밥은 \n이렇게 \n기름이 좔좔 흘러줘야지 \n더군다나 삼겹살김치볶음밥인데 \nㅋㅋㅋ 짝꿍은 결국 다 못먹고 남겼다 \n양이 많아서\n전자레인지 사용가능 용기라 \n나중에 그냥 전자레인지에\n간편하게 돌려서 먹으면 된다 \n귿\n끗  #홍대돈떡 #홍대배달 #홍대배달전문 #홍대배달음식 #홍대배달맛집 #배달맛집 #삼겹살김치볶음밥 #치킨마요덮답'}, 2: {'title': '[맛집/고터] 김치볶음밥이 맛있는 레트로 맛집 『하트타임』', 'nickname': '핑림', 'datetime': '2020. 12. 10. 23:26', 'c

In [18]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')

# 엑셀로 저장하기
result_df.to_excel("blog_content.xlsx")